<a href="https://colab.research.google.com/github/rgprado/Mineracao_de_Texto_na_Web_Coleta_de_Dados/blob/main/Atividade_1_Coleta_de_dados_na_Web%2C_Pr%C3%A9_processamento_e_Extra%C3%A7%C3%A3o_de_Caracter%C3%ADsticas_do_Texto_Rodrigo_Prado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4
!conda activate NLP
!pip install nltk
!pip install --upgrade gensim
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: conda: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 4.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=311ecd897a6f357df24f239e3a2d88976b01be6761af89d52c0f70742020e7e4
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn


In [64]:
import pandas as pd
import numpy as np
import json
from typing import List
import os.path
from os import path

import os
import glob

from bs4 import BeautifulSoup
import requests
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def create_json_directory(path: str):
  output_path = "output"

  if not os.path.exists(output_path):
    os.makedirs(output_path)

  if not os.path.exists(output_path + '/' + path):
    os.makedirs(output_path + '/' + path)
  else:
    print("Path already exists")

In [4]:
def read_page_get_links(keywords: str) -> list:
    response = requests.get("https://repositorio.ufpe.br/simple-search?location=123456789%2F50&query={}".format(keywords))
    soup = BeautifulSoup(response.text.strip(), 'html.parser')
    tags = soup.find_all(name='table')
    tag_table = tags[0]
    tags_a = tag_table.find_all(name='a')
    
    urls = []
    
    for i in range(0, len(tags_a), 2): 
        link = tags_a[i]["href"]
        urls.append(link)
    
    return urls

In [5]:
def read_table_get_title_and_abstract(info_table: List[str]) -> dict:
    all_titles = info_table.find_all(name='td', attrs={'class': 'dc_title'})
    title = all_titles[-1].contents[0]

    all_abst = info_table.find_all(name='td', attrs={'class': 'dc_description_abstract'})
    abst = all_abst[-1].contents[0]

    dict_thesis_info = {}
    dict_thesis_info = {
        'title': title,
        'abstract': abst,
    }

    return dict_thesis_info

In [6]:
def create_tables(urls):
  info_table = []

  for url in urls:
      response = requests.get(f'https://repositorio.ufpe.br/{url}', allow_redirects=True)
      soup = BeautifulSoup(response.text.strip(), 'html.parser')
      tags_table = soup.find_all(name='table', attrs={'class': 'itemDisplayTable'})
      info_table.append(tags_table[0])

  return info_table

In [7]:
def create_output_folder(topic: str, info_table):
  counter = 1

  create_json_directory(topic)

  for table in info_table:
      output = read_table_get_title_and_abstract(table)
      
      # Criar a estrutura de pastas antes de executar a celula
      with open(f"./output/{topic}/thesis{counter}.json", 'w') as json_file:
          json.dump(output, json_file, ensure_ascii=False) #TODO: Automatizar a criação das pastas
          
      counter = counter + 1

In [8]:
topic_list = ["machine+learning", "economia", "estatistica", "historia", "neuro+ciencia"]
all_urls = []

for topic in topic_list:
  urls = read_page_get_links(topic)
  info_table = create_tables(urls)
  create_output_folder(topic, info_table)

In [9]:
path = os.getcwd()
files = glob.glob(os.path.join(path, "output/*/*.json"))

In [10]:
jsons = [pd.read_json(file, lines=True) for file in files]
thesis = pd.concat(jsons, ignore_index=True)

thesis["subject"] = [os.path.split(os.path.dirname(file))[-1] for file in files]

In [11]:
thesis

,title,abstract,subject
0,Visão computacional aplicada a gestão da manut...,O desgaste progressivo de peças rotativas como...,machine+learning
1,Um método de aprendizagem seqüencial com filtr...,"Em aplicações de aprendizagem de máquina, é co...",machine+learning
2,Development of machine and deep learning based...,Artificial intelligence-based algorithms have ...,machine+learning
3,Social machines: a unified paradigm to describ...,"The open, distributed approach of the Web and ...",machine+learning
4,Malware detection in macOS using supervised le...,The development of macOS malware has grown sig...,machine+learning
5,Reconhecimento de espécies de mosquitos por me...,As doenças transmitidas por mosquitos vetores ...,machine+learning
6,Time Aware Sigmoid Optimization : a new learni...,The correct choice of hyperparameters for the ...,machine+learning
7,Time series forecasting with deep forest regre...,A time series is a collection of ordered obser...,machine+learning
8,SMADL – The social machines architecture descr...,We are experiencing a high growth in the numbe...,machine+learning
9,A machine learning-based methodology for autom...,Oil refineries process hazardous substances at...,machine+learning


In [12]:
list_stop_en = stopwords.words('english')
print('English Stop Words', list_stop_en)
list_stop_pt = stopwords.words('portuguese')
print('Portuguse Stop Words', list_stop_pt)

English Stop Words ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'sam

In [13]:
texto = ''
for doc in thesis[:1].iterrows():
  texto = doc[1]['title'] + ' ' + doc[1]['abstract']

print(texto)
print(thesis[:1]['abstract'])

Visão computacional aplicada a gestão da manutenção de sistemas de engrenagens O desgaste progressivo de peças rotativas como engrenagens é um problema recorrente,
principalmente para sistemas de produção cada vez mais sensíveis as variações destes
componentes. Por isso, este trabalho propõe um framework para detectar desgastes superficiais
em engrenagens de dentes retos, por meio de imagens digitais superficiais de dentes de
engrenagem. Com o objetivo de responder a quatro questões norteadoras sobre o tema visão
computacional e gestão da manutenção, uma revisão sistemática foi elaborada por meio de
pesquisas realizadas nas bases de dados IEEE, Scopus, ScienceDirect e Web of Science e o
software Start foi utilizado no processo metodológico. Uma bancada experimental foi
construída para permitir a captura de imagens de dentes de engrenagem por meio de câmeras
endoscópicas. As imagens foram selecionadas, pré-processadas, armazenadas em um banco de
dados e utilizadas no estudo experimental

In [37]:
documents_tokens = []
vocabulary = set()
for article in thesis.iterrows():
  doc = article[1]['title'] + ' ' + article[1]['abstract']
  doc_tokens = [
    token.lower() # pre processamento: todas as letras ficam minúsculas
      for token in word_tokenize(doc, language='portuguese') # tokenizador para o idioma inglês a nível de palavras
        if not token.lower() in list_stop_pt and token.isalnum() # excluir tokens na lista "stop" e símbolos
  ]

  # doc_tokens = []
  # for token in word_tokenize(doc, language='english'):
  #   token = token.lower()
  #   if not token in list_stop:
  #     if not token.isalnum():
  #       doc_tokens.append(token)

  vocabulary.update(set(doc_tokens))
  documents_tokens.append(doc_tokens)

In [38]:
len(vocabulary)

3608

In [39]:
for v in list(vocabulary):
  if len(v) == 1:
    print(v)

g
4
n
s
x
1
8
b
t
9
c
i
p
á
2
7
3
k
d
5


In [46]:
documents_tokens[0]

['visão',
 'computacional',
 'aplicada',
 'gestão',
 'manutenção',
 'sistemas',
 'engrenagens',
 'desgaste',
 'progressivo',
 'peças',
 'rotativas',
 'engrenagens',
 'problema',
 'recorrente',
 'principalmente',
 'sistemas',
 'produção',
 'cada',
 'vez',
 'sensíveis',
 'variações',
 'destes',
 'componentes',
 'trabalho',
 'propõe',
 'framework',
 'detectar',
 'desgastes',
 'superficiais',
 'engrenagens',
 'dentes',
 'retos',
 'meio',
 'imagens',
 'digitais',
 'superficiais',
 'dentes',
 'engrenagem',
 'objetivo',
 'responder',
 'quatro',
 'questões',
 'norteadoras',
 'sobre',
 'tema',
 'visão',
 'computacional',
 'gestão',
 'manutenção',
 'revisão',
 'sistemática',
 'elaborada',
 'meio',
 'pesquisas',
 'realizadas',
 'bases',
 'dados',
 'ieee',
 'scopus',
 'sciencedirect',
 'web',
 'of',
 'science',
 'software',
 'start',
 'utilizado',
 'processo',
 'metodológico',
 'bancada',
 'experimental',
 'construída',
 'permitir',
 'captura',
 'imagens',
 'dentes',
 'engrenagem',
 'meio',
 'câme

In [54]:
print(thesis[:1])
pre_processed_documents = [' '.join(doc) for doc in documents_tokens]
print(pre_processed_documents[0])

                                               title  \
0  Visão computacional aplicada a gestão da manut...   

                                            abstract           subject  
0  O desgaste progressivo de peças rotativas como...  machine+learning  
visão computacional aplicada gestão manutenção sistemas engrenagens desgaste progressivo peças rotativas engrenagens problema recorrente principalmente sistemas produção cada vez sensíveis variações destes componentes trabalho propõe framework detectar desgastes superficiais engrenagens dentes retos meio imagens digitais superficiais dentes engrenagem objetivo responder quatro questões norteadoras sobre tema visão computacional gestão manutenção revisão sistemática elaborada meio pesquisas realizadas bases dados ieee scopus sciencedirect web of science software start utilizado processo metodológico bancada experimental construída permitir captura imagens dentes engrenagem meio câmeras endoscópicas imagens selecionadas armazenadas b

In [19]:
custom_list_stop = []
for v in list(vocabulary):
  if len(v) == 1 or len(v) == 2:
    custom_list_stop.append(v)

custom_list_stop = custom_list_stop + ['xterm', 'x11r5', 'x11', 'xlib', 'vax', 'uwo', 'pl8', 'pl9']
print(custom_list_stop)

del documents_tokens
del vocabulary
documents_tokens = []
vocabulary = set()
for doc in pre_processed_documents:
    tokens = [token for token in word_tokenize(doc, language='portuguese') if not token in custom_list_stop]
    documents_tokens.append(tokens)
    vocabulary.update(set(tokens))

print('Tamanho do novo vocabulário: ', len(vocabulary))

del pre_processed_documents
pre_processed_documents = [' '.join(doc) for doc in documents_tokens]

['ee', 'f1', 'g', '95', 'on', '14', '1º', '4', '72', 'n', '10', '2º', '19', '17', 'is', 's', 'in', 'x', 'dá', 'we', '1', 'pn', '65', '8', '11', 'gc', '15', 'la', 'b', 'cd', '52', 't', 'wi', 'rs', 'ii', 'sm', '9', '50', 'c', 'i', 'mm', 'ir', 'dp', 'je', '70', '12', '56', 'dl', 'xx', 'an', '78', 'nº', '90', 'ml', 'ar', 'p', 'á', '81', '18', 'ia', '2', '20', 'mg', '7', 'at', '98', 'qm', 'aí', 'ex', '32', 'it', '30', 't2', '80', 'ph', '3', '48', 'k', '42', 'fe', 'si', 'to', '85', '60', 'lm', 'be', 'up', 'd', 'ps', 'es', 'pe', 'of', '16', '5', 'by', 'xterm', 'x11r5', 'x11', 'xlib', 'vax', 'uwo', 'pl8', 'pl9']
Tamanho do novo vocabulário:  3513


In [20]:
for v in list(vocabulary):
  if len(v) == 3:
    print(v)

lei
rbf
ktk
für
how
pca
sim
one
any
vez
adl
têm
vai
mês
ter
tcm
dsl
dar
boa
aim
631
the
can
two
few
but
adu
lee
mão
uso
hal
pib
cpe
deu
oil
ide
kdd
fwd
hiv
bpn
bom
way
and
ano
has
vgg
sul
955
may
iii
all
sob
diz
bad
luz
vêm
vão
not
cnn
too
via
ips
tal
fez
rio
mlp
dor
snc
svm
was
end
rna
águ
knn
imc
out
faz
fim
are
gal
dão
200
dez
bem
dia
vem
rel
edc
142
new
snh
set
avd
180
klm
elm
shi
car
sgd
web
its
ies


In [55]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(stop_words=list_stop_pt, min_df=0.005, max_df=0.5, max_features=3500)
count_vect.fit(pre_processed_documents)

CountVectorizer(max_df=0.5, max_features=3500, min_df=0.005,
                stop_words=['a', 'à', 'ao', 'aos', 'aquela', 'aquelas',
                            'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até',
                            'com', 'como', 'da', 'das', 'de', 'dela', 'delas',
                            'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é',
                            'ela', 'elas', 'ele', 'eles', ...])

In [57]:
print('Tamanho do vocabulário: ', len(count_vect.vocabulary_))
counter = 0
for _, count in count_vect.vocabulary_.items():
    counter += count
    
print('Quantidade total de palavras: ', counter)

Tamanho do vocabulário:  3500
Quantidade total de palavras:  6123250


In [58]:
sorted(count_vect.vocabulary_.items(), reverse=True)

[('único', 3499),
 ('única', 3498),
 ('últimos', 3497),
 ('última', 3496),
 ('ótica', 3495),
 ('órgão', 3494),
 ('óbitos', 3493),
 ('índices', 3492),
 ('índice', 3491),
 ('ênfase', 3490),
 ('êmico', 3489),
 ('ética', 3488),
 ('âmbitos', 3487),
 ('âmbito', 3486),
 ('áudio', 3485),
 ('árvore', 3484),
 ('áreas', 3483),
 ('área', 3482),
 ('água', 3481),
 ('águ', 3480),
 ('zoologia', 3479),
 ('zika', 3478),
 ('years', 3477),
 ('yahr', 3476),
 ('xx', 3475),
 ('xukuru', 3474),
 ('xprotect', 3473),
 ('wraps', 3472),
 ('would', 3471),
 ('works', 3470),
 ('work', 3469),
 ('words', 3468),
 ('without', 3467),
 ('with', 3466),
 ('wi', 3465),
 ('whole', 3464),
 ('while', 3463),
 ('which', 3462),
 ('where', 3461),
 ('when', 3460),
 ('were', 3459),
 ('weights', 3458),
 ('webqda', 3457),
 ('web', 3456),
 ('we', 3455),
 ('ways', 3454),
 ('way', 3453),
 ('was', 3452),
 ('vômitos', 3451),
 ('vírus', 3450),
 ('vídeo', 3449),
 ('vêm', 3448),
 ('vão', 3447),
 ('vários', 3446),
 ('várias', 3445),
 ('vozes', 3

In [59]:
count_vect.vocabulary_.items()

dict_items([('visão', 3433), ('computacional', 652), ('aplicada', 239), ('gestão', 1544), ('manutenção', 1987), ('sistemas', 3043), ('engrenagens', 1235), ('desgaste', 961), ('progressivo', 2630), ('peças', 2452), ('rotativas', 2926), ('problema', 2590), ('recorrente', 2777), ('principalmente', 2579), ('produção', 2615), ('cada', 473), ('vez', 3408), ('sensíveis', 2990), ('variações', 3383), ('destes', 977), ('componentes', 633), ('trabalho', 3269), ('propõe', 2654), ('framework', 1478), ('detectar', 982), ('desgastes', 962), ('superficiais', 3131), ('dentes', 918), ('retos', 2898), ('meio', 2027), ('imagens', 1674), ('digitais', 1028), ('engrenagem', 1234), ('objetivo', 2249), ('responder', 2880), ('quatro', 2713), ('questões', 2723), ('norteadoras', 2222), ('sobre', 3057), ('tema', 3182), ('revisão', 2910), ('sistemática', 3045), ('elaborada', 1169), ('pesquisas', 2447), ('realizadas', 2749), ('bases', 414), ('dados', 867), ('ieee', 1662), ('scopus', 2956), ('sciencedirect', 2955), (

In [60]:
X = count_vect.transform(pre_processed_documents)

In [61]:
X.shape

(50, 3500)

In [63]:
print(X[0,:])

  (0, 0)	1
  (0, 56)	1
  (0, 59)	1
  (0, 67)	1
  (0, 107)	1
  (0, 117)	2
  (0, 164)	2
  (0, 165)	1
  (0, 187)	1
  (0, 212)	1
  (0, 239)	1
  (0, 242)	1
  (0, 301)	1
  (0, 388)	1
  (0, 402)	1
  (0, 403)	1
  (0, 414)	1
  (0, 447)	2
  (0, 473)	1
  (0, 489)	1
  (0, 490)	1
  (0, 491)	1
  (0, 560)	2
  (0, 571)	2
  (0, 594)	1
  :	:
  (0, 2956)	1
  (0, 2972)	1
  (0, 2990)	1
  (0, 3043)	2
  (0, 3045)	1
  (0, 3057)	1
  (0, 3072)	1
  (0, 3097)	1
  (0, 3131)	2
  (0, 3132)	1
  (0, 3136)	1
  (0, 3148)	1
  (0, 3173)	1
  (0, 3182)	1
  (0, 3213)	1
  (0, 3269)	1
  (0, 3305)	1
  (0, 3316)	1
  (0, 3361)	1
  (0, 3362)	1
  (0, 3383)	1
  (0, 3388)	1
  (0, 3408)	1
  (0, 3433)	2
  (0, 3456)	1


In [67]:
tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(X)

TfidfTransformer()

In [68]:
X_tfidf = tfidf_transformer.transform(X)
print(X_tfidf[0,:])

  (0, 3456)	0.05788243034020662
  (0, 3433)	0.11576486068041324
  (0, 3408)	0.05126302577778832
  (0, 3388)	0.05788243034020662
  (0, 3383)	0.06919837718168405
  (0, 3362)	0.05423950913168312
  (0, 3361)	0.042923562290205705
  (0, 3316)	0.04136757972493561
  (0, 3305)	0.06257897261926575
  (0, 3269)	0.02863113209483346
  (0, 3213)	0.04874644495999762
  (0, 3182)	0.05423950913168312
  (0, 3173)	0.06919837718168405
  (0, 3148)	0.06257897261926575
  (0, 3136)	0.05788243034020662
  (0, 3132)	0.05788243034020662
  (0, 3131)	0.1383967543633681
  (0, 3097)	0.06257897261926575
  (0, 3072)	0.05126302577778832
  (0, 3057)	0.030811093556101888
  (0, 3045)	0.05788243034020662
  (0, 3043)	0.09749288991999525
  (0, 2990)	0.06919837718168405
  (0, 2972)	0.06919837718168405
  (0, 2956)	0.06919837718168405
  :	:
  (0, 594)	0.06919837718168405
  (0, 571)	0.1383967543633681
  (0, 560)	0.1383967543633681
  (0, 491)	0.06257897261926575
  (0, 490)	0.06919837718168405
  (0, 489)	0.06919837718168405
  (0, 473

In [69]:
X_tfidf.shape

(50, 3500)

In [71]:
# Pre processamento
final_list_stop_words = list_stop_pt + custom_list_stop
query = 'Economia digital na era do machine learning deep learning, inteligência artificial, um panorama histórico no brasil e no mundo'

query_tokens = [token for token in word_tokenize(query, language='portuguese') if not token in final_list_stop_words]
print(query_tokens)
pre_processed_query = ' '.join(query_tokens)
print(pre_processed_query)

# Extração de características de contagem
X_query = count_vect.transform([pre_processed_query])
print('1a etapa de extração de características')
print(X_query)

# Extração de características de frequência TF-IDF
X_tfidf_query = tfidf_transformer.transform(X_query)
print('2a etapa de extração de características')
print(X_tfidf_query)

['Economia', 'digital', 'machine', 'learning', 'deep', 'learning', ',', 'inteligência', 'artificial', ',', 'panorama', 'histórico', 'brasil', 'mundo']
Economia digital machine learning deep learning , inteligência artificial , panorama histórico brasil mundo
1a etapa de extração de características
  (0, 310)	1
  (0, 453)	1
  (0, 890)	1
  (0, 1029)	1
  (0, 1136)	1
  (0, 1609)	1
  (0, 1787)	1
  (0, 1896)	2
  (0, 1963)	1
  (0, 2140)	1
2a etapa de extração de características
  (0, 2140)	0.37755449395507024
  (0, 1963)	0.25407221713406597
  (0, 1896)	0.4871617082012228
  (0, 1787)	0.29593714847989816
  (0, 1609)	0.3158133555445681
  (0, 1136)	0.21795596641208764
  (0, 1029)	0.3158133555445681
  (0, 890)	0.27969710482258975
  (0, 453)	0.25407221713406597
  (0, 310)	0.27969710482258975


In [76]:
sorted_X_indices = np.argsort(X.toarray(), axis=1)[:, ::-1]
sorted_Xtfidf_indices = np.argsort(X_tfidf.toarray(), axis=1)[:,::-1]

print('Termos mais comuns nos 10 primeiros documentos:')
for i in range(10):
  print(count_vect.get_feature_names_out()[sorted_X_indices[i, :3]])

print('\nTermos mais relevantes nos 10 primeiros documentos:')
for i in range(10):
  print(count_vect.get_feature_names_out()[sorted_Xtfidf_indices[i, :3]])

print('\nTermos mais relevantes na base de dados:')
mean_Xtfidf = X_tfidf.mean(axis=0)
sorted_meanXtfidf_indices = np.argsort(mean_Xtfidf)[::-1]
print(count_vect.get_feature_names_out()[sorted_meanXtfidf_indices[:3]])

Termos mais comuns nos 10 primeiros documentos:
['engrenagens' 'dentes' 'imagens']
['método' 'aprendizagem' 'sequencial']
['the' 'to' 'and']
['and' 'the' 'social']
['the' 'macos' 'of']
['mosquitos' 'doenças' 'vetores']
['the' 'training' 'of']
['to' 'of' 'the']
['the' 'of' 'to']
['of' 'the' 'and']

Termos mais relevantes nos 10 primeiros documentos:
['engrenagens' 'dentes' 'framework']
['sequencial' 'método' 'treinamento']
['the' 'and' 'to']
['and' 'the' 'social']
['macos' 'the' 'malware']
['mosquitos' 'vetores' 'doenças']
['the' 'training' 'of']
['to' 'of' 'the']
['the' 'of' 'to']
['of' 'oil' 'the']

Termos mais relevantes na base de dados:
[['because' 'under' 'mainly' ... 'economia' 'história' 'the']]


In [75]:
print(len(sorted_X_indices))
sorted_X_indices[0]

50


array([1235,  918, 1674, ..., 2300, 2299, 1749])